In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import os

# =======================================
# ⚡ 1. Device setup (GPU if available)
# =======================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =======================================
# 🔸 2. Load your trained PyTorch model
# =======================================
model_path = "models/fruit_classifier_resnet50_E20.pth"

# Recreate the same architecture used in training
model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # 5 classes (adjust if needed)

# Load weights
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# =======================================
# 🔸 3. Define preprocessing transforms
# =======================================
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# =======================================
# 🔸 4. Class names
# =======================================
class_names = ['Apple', 'Banana', 'Grapes', 'Mango', 'Orange']  # adjust as needed

# =======================================
# 🔸 5. Loop through all images in folder
# =======================================
img_folder = "Img"  # your image folder
valid_extensions = (".png", ".jpg", ".jpeg", ".bmp")

for filename in os.listdir(img_folder):
    if filename.lower().endswith(valid_extensions):
        img_path = os.path.join(img_folder, filename)
        img = Image.open(img_path).convert("RGB")

        # Preprocess
        img_tensor = preprocess(img).unsqueeze(0).to(device)

        # Predict
        with torch.no_grad():
            outputs = model(img_tensor)
            probabilities = torch.softmax(outputs, dim=1)
            confidence, predicted_class = torch.max(probabilities, 1)

        predicted_label = class_names[predicted_class.item()]
        conf = confidence.item() * 100

        print(f"📁 File Name: {filename}")
        print(f"   🏷️ Predicted Class: {predicted_label}")
        print(f"   🎯 Confidence: {conf:.2f}%")
        print("-" * 40)


Using device: cpu
📁 File Name: 91PD3DRbUaL._SL1500_.jpg
   🏷️ Predicted Class: Mango
   🎯 Confidence: 100.00%
----------------------------------------
📁 File Name: banana-cavendish_0.png
   🏷️ Predicted Class: Banana
   🎯 Confidence: 97.37%
----------------------------------------
📁 File Name: Bunch-of-bananas-00871a2.jpg
   🏷️ Predicted Class: Banana
   🎯 Confidence: 100.00%
----------------------------------------
📁 File Name: gettyimages-726875543-2000-bd10c8d62fcd4f6dbf8ff1da5ffc3371.jpg
   🏷️ Predicted Class: Mango
   🎯 Confidence: 100.00%
----------------------------------------
📁 File Name: mandarin-orange.jpg
   🏷️ Predicted Class: Orange
   🎯 Confidence: 93.61%
----------------------------------------
📁 File Name: pngtree-delicious-black-grapes-png-image_14877346.png
   🏷️ Predicted Class: Grapes
   🎯 Confidence: 99.90%
----------------------------------------


c:\Python314\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
